In [1]:
import tkinter as tk

In [2]:
board_size = 8

# Inicializar los colores de las casillas
white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
board[3][3] = board[4][4] = 'W'
board[3][4] = board[4][3] = 'B'
print(board)

[[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', 'W', 'B', ' ', ' ', ' '], [' ', ' ', ' ', 'B', 'W', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


In [3]:
points_direction = {
    0: 'N',
    1: 'NE',
    2: 'E',
    3: 'SE',
    4: 'S',
    5: 'SO',
    6: 'O',
    7: 'NO'
}
directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def check_mov_direction(row, col,player):
    pos=0
    direction_check=[]
    check =False
    if player == "black":
        openent= 'W'
    else:
        openent= 'B'
    for dr, dc in directions:
        if(board[row+dr][col+dc]==openent):
            direction_check.append(points_direction[pos])
            
            check= True
        pos+=1   
    print(direction_check)    
    return direction_check,check
def check_mov(row, col,direction_check, player):
    coor_repaint=[]
    if player == "black":
        openent= 'W'
    else:
        openent= 'B'
    for dir in direction_check:
        y,x=row, col
        print("La direccion",dir, "filas y col",row, col)
        if dir == 'N':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y-=1
               x+=1 
        elif dir == 'NE':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y-=1
               x+=1  
        elif dir == 'E':
            while board[y][x] == openent or  board[y][x] == ' ' :
               coor_repaint.append((y,x)) 
               x+=1 
        elif dir == 'SE':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y+=1
               x+=1 
        elif dir == 'S':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y+=1
                
        elif dir == 'SO':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y+=1
               x-=1 
        elif dir == 'O':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               x-=1
        elif dir == 'NO':
            while board[y][x] == openent or  board[y][x] == ' ':
               coor_repaint.append((y,x)) 
               y-=1
               x-=1 
        else:
            pass 
    print("LAs coor a repintar son", coor_repaint)       
    return coor_repaint

In [4]:


# Tamaño del tablero (8x8)

# Función para crear una casilla en el tablero
def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
    button.grid(row=row + 1, column=col + 1)
    return button

# Función para pintar una casilla
def pintar_casilla(row, col):
    if board[row][col] == ' ':
        color = white_color if current_color == 'W' else black_color
        direction_check,check= check_mov_direction(row, col, color)
        print("ES el turno de",color)
        if check:
            coor_repaint= check_mov(row, col,direction_check, color)
           
            for y,x in coor_repaint:
                board[y][x] = current_color
                crear_casilla(tablero, y, x, color)
            imprimir_coordenadas(row, col)  
            cambiar_color()
# Función para cambiar el color actual
def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")

root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()

current_color = 'B'

label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()

def darw_map():
    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    # Crear los botones del tablero
    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            else:
                crear_casilla(tablero, row, col)

darw_map()
root.mainloop()


['S']
ES el turno de black
La direccion S filas y col 2 3
LAs coor a repintar son [(2, 3), (3, 3)]
Posición seleccionada: (2, 3)
['NE', 'E']
ES el turno de white
La direccion NE filas y col 4 2


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\miky1\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\2304341914.py", line 5, in <lambda>
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
                                                                      ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\2304341914.py", line 16, in pintar_casilla
    coor_repaint= check_mov(row, col,direction_check, color)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\1159174302.py", line 44, in check_mov
    while board[y][x] == openent or  board[y][x] == ' ':
          ~~~~~~~~^^^
IndexError: list index out of range


['NE', 'E']
ES el turno de white
La direccion NE filas y col 4 2


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\miky1\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\2304341914.py", line 5, in <lambda>
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
                                                                      ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\2304341914.py", line 16, in pintar_casilla
    coor_repaint= check_mov(row, col,direction_check, color)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miky1\AppData\Local\Temp\ipykernel_7248\1159174302.py", line 44, in check_mov
    while board[y][x] == openent or  board[y][x] == ' ':
          ~~~~~~~~^^^
IndexError: list index out of range


In [5]:
points_direction[1]

'NE'

In [6]:
board[2][3]

'B'